In [38]:
import numpy as np
import pandas as pd
from itertools import combinations
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sc
import time
import sys

In [39]:
data = pd.read_csv('week_data_2018.csv',index_col=0)
data = data.drop(['game_id', 'home_team'], axis=1)
data_plt = np.array(data)
print(data.head())
print(np.shape(data))

   winning_team  losing_team  diff_abs  week
1             1           18       3.5     1
2             2           24      41.5     1
3            22            9       2.5     1
4            17            8      12.5     1
5             7           20       4.5     1
(256, 4)


In [40]:
def week_UB_Pij(data, weekub, method, weighted = True):
    
    # Getting the Data in the right form
    data = data.drop(['game_id', 'home_team'], axis=1)
    if method == "less":
        # IF METHOD IS WEEKS LESS THAN:
        data["week"] = data[data["week"] < weekub]["week"]
    if method == "equal":
        # IF METHOD IS GETTING THE EXACT WEEK
        data["week"] = data[data["week"] == weekub]["week"]
    if method == "between":
        # If METHOD IS BETWEEN (WITH CONTAINMENT ON THE UPPER END)
        data["week"] = data[np.logical_and(data["week"] > weekub[0],data["week"] <= weekub[1])]["week"]
    data = data.dropna()
    data = data.drop("week", axis = 1)
    data_np = np.array(data)
    
    Pij = np.zeros((32,32))
    
    # Computing the Pij Matrix
    for iCnt in range(np.shape(data_np)[0]):
        winner = int(data_np[iCnt,0])-1
        loser =  int(data_np[iCnt,1])-1
        # checking if an interdivisional game has occurred
        if (Pij[loser,winner] == 0) & (Pij[winner,loser] == 0):
            Pij[winner,loser] = data_np[iCnt,2]
        # when the other team had won the first game
        elif (Pij[loser,winner] > 0):
            if (Pij[loser,winner] > data_np[iCnt,2]):
                Pij[loser,winner] = (Pij[loser,winner] - data_np[iCnt,2])/2
            else: 
                Pij[winner,loser] = -(Pij[loser,winner] - data_np[iCnt,2])/2
                Pij[loser,winner] = 0
        # when the same team won the first game
        else: 
            Pij[winner,loser] = (Pij[winner,loser] + data_np[iCnt,2])/2
            
    if(weighted == False):
                Pij[Pij!=0] = 1
            
    # making sure none of the columns sum to 0
    if len(np.where(Pij.sum(axis = 0) == 0)[0]) > 0:
        empty_vect = np.where(Pij.sum(axis = 0) == 0)[0]
        for iCnt in range(len(empty_vect)):
            Pij[empty_vect[iCnt],empty_vect[iCnt]] = 1
    
    # making the Pij matrix
    Pij = Pij/Pij.sum(axis=0)
        
    # returning the Pij Matrix
    return Pij

# Determining Which Weeks to Iterate Over

In [101]:
blocks = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
data = pd.read_csv('week_data_2018.csv',index_col=0)
alpha = 0.1

# Computing the Scores

In [102]:
# making the teleporting probabilities uniform TO START
v = np.ones(32)/32
v = v[np.newaxis]
v = v.T


for iCnt in range(len(blocks)-1):
    # setting the weeks to look at
    weekub = [blocks[iCnt],blocks[iCnt+1]]
    
    # setting Pij
    Pij = week_UB_Pij(data, weekub, method = "between", weighted = True)
    # If we want the teleportation probabilties unifrom,
    # or based on team winning perc
                
    # initializing our first guess
    x0 = np.ones(32)/32
    x0 = x0[np.newaxis]
    x0 = x0.T
            
    # performing the simulation
    for i in range(5000):
        x0 = alpha*np.dot(Pij,x0) + (1-alpha)*v
        
    v = x0

    
scores = np.squeeze(x0)
         
    
    

In [103]:
teams = pd.read_csv('teams_2018.csv',index_col=0)
team_names = np.array(teams["team_name"])
print(team_names)

['PHI' 'BAL' 'NYG' 'NO' 'NE' 'MIN' 'MIA' 'IND' 'CLE' 'LAC' 'DEN' 'CAR'
 'ARI' 'GB' 'DET' 'OAK' 'CIN' 'ATL' 'WAS' 'TEN' 'TB' 'PIT' 'NYJ' 'BUF' 'SF'
 'LA' 'JAX' 'DAL' 'CHI' 'HOU' 'KC' 'SEA']


In [104]:
print(scores)

[ 0.02691708  0.03881545  0.02104008  0.06493293  0.0480177   0.03483686
  0.02078921  0.0367972   0.03321659  0.04371973  0.02117981  0.02179533
  0.0126884   0.02361179  0.02093287  0.0183264   0.01731072  0.02280294
  0.02064719  0.04698229  0.02534928  0.03728661  0.01428251  0.01283474
  0.0181206   0.03551699  0.01941356  0.0424238   0.05913507  0.04906654
  0.05062677  0.04058296]


In [105]:
teamrank = {'Team': team_names, 'score': scores, 'wins': 32*16/2*scores}
teamrank_df = pd.DataFrame(data=teamrank)
teamrank_df = teamrank_df.sort_values(by=['score'], ascending = False)
print(teamrank_df)

   Team     score       wins
3    NO  0.064933  16.622830
28  CHI  0.059135  15.138577
30   KC  0.050627  12.960454
29  HOU  0.049067  12.561035
4    NE  0.048018  12.292531
19  TEN  0.046982  12.027466
9   LAC  0.043720  11.192250
27  DAL  0.042424  10.860492
31  SEA  0.040583  10.389237
1   BAL  0.038815   9.936754
21  PIT  0.037287   9.545371
7   IND  0.036797   9.420083
25   LA  0.035517   9.092349
5   MIN  0.034837   8.918235
8   CLE  0.033217   8.503448
0   PHI  0.026917   6.890772
20   TB  0.025349   6.489416
13   GB  0.023612   6.044618
17  ATL  0.022803   5.837554
11  CAR  0.021795   5.579604
10  DEN  0.021180   5.422032
2   NYG  0.021040   5.386261
14  DET  0.020933   5.358815
6   MIA  0.020789   5.322039
18  WAS  0.020647   5.285680
26  JAX  0.019414   4.969872
15  OAK  0.018326   4.691559
24   SF  0.018121   4.638874
16  CIN  0.017311   4.431545
22  NYJ  0.014283   3.656323
23  BUF  0.012835   3.285695
12  ARI  0.012688   3.248231
